# Refactor Code For Scraping Monster

In [1]:
import pandas as pd # For Data Storage
import requests # For website connection
from bs4 import BeautifulSoup # For HTML parsing
import time # For sleep 
import re # Regular expressions for removing non-ascii terms
from itertools import groupby # For removing duplicates from lists
import math # Need ceiling expression
from nltk.corpus import stopwords # For filtering out words like 'is', 'the', 'of'
stop_words = set(stopwords.words("english")) #initialize stopwords    
import matplotlib.pyplot as plt

# Initialize Empty Lists For Dictionary 

In [90]:
#Initialize Empty State Set
state = []
job_title = []

#Initialize Empty Skills List
r = []
python = []
java = []
cpp = []
ruby = []
perl = []
matlab = []
javascript = []
scala = []
excel = []
vba = []
tableau = []
d3 = []
hadoop = []
mapreduce = []
spark = []
pig = []
hive = []
shark = []
oozie = []
zookeeper = []
flume = []
mahout = []
sql = [] 
mysql = []
oracle = []
postgresql = []
nosql =[]
sybase = []
rethinkdb = []
memcacheddb = []
couchdb = []
hbase = []
cassandra = []
mongodb = []
db2 = []

skills = ['r',
'python',
'java',
'c++',
'ruby',
'perl',
'matlab',
'javascript',
'scala', 
'excel',
'vba',
'tableau',
'd3',
'hadoop',
'mapreduce',
'spark',
'pig',
'hive',
'shark',
'oozie',
'zookeeper',
'flume',
'mahout',
'sql', 
'mysql',
'oracle',
'postgresql',
'nosql',
'sybase',
'rethinkdb',
'memcacheddb',
'couchdb',
'hbase',
'cassandra',
'mongodb',
'db2'
]

## Initialize the Dictionary 

In [92]:
dictionary = {'job_title':job_title,
     'state':state,
     'r':r,
     'python':python,
     'java':java,
     'c++':cpp,
     'ruby':ruby,
     'perl':perl,
     'matlab':matlab,
     'javascript':javascript,
     'scala':scala,
     'excel':excel,
     'vba':vba,
     'tableau':tableau,
     'd3':d3,
     'hadoop':hadoop,
     'mapreduce':mapreduce,
     'spark':spark,
     'pig':pig,
     'hive':hive,
     'shark':shark,
     'oozie':oozie,
     'zookeeper':zookeeper,
     'flume':flume,
     'mahout':mahout,
     'sql':sql, 
     'mysql':mysql,
     'oracle':oracle,
     'postgresql':postgresql,
     'nosql':nosql,
     'sybase':sybase,
     'rethinkdb':rethinkdb,
     'memcacheddb':memcacheddb,
     'couchdb':couchdb,
     'hbase':hbase,
     'cassandra':cassandra,
     'mongodb':mongodb,
     'db2':db2}

In [51]:
def addValsToDict(url_state,url_job_title):
    #Append state
    state.append(url_state)

    #Append job_title
    job_title.append(url_job_title)

    #Check if the each skill is in the wrangled text
    for skill in skills:
        if (skill in text_final) == True:
            dictionary[skill].append(1)
        else:
            dictionary[skill].append(0)

In [93]:
def clearDict():
    for key in dictionary:
        dictionary[key] = []

## Get a URL for a Job Search for a chosen State

In [52]:
url_state = 'NY'

### Store the job search in a beautiful soup object

In [98]:
url = 'https://www.monster.com/jobs/search/?q=data-scientist&where='+url_state+'&stpage=1'

In [54]:
#Access the website
response = requests.get(url)
##print(response)  #Response 200 means the get request was a success
soup = BeautifulSoup(response.text, "html.parser")

# Get the url for up to 250 job posts in each state

In [99]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [111]:
# Initialize Empty URL Dictionary
final_state_url_dict = {}

for url_state in states:
    url = 'https://www.monster.com/jobs/search/?q=data-scientist&where='+url_state+'&stpage=1'
    response = requests.get(url) #Get the html code
    soup = BeautifulSoup(response.text, "html.parser") #Store html in a soup object 

    #Get the total number of listed jobs 
    totalJobOpenings = float(re.sub('[^0-9]','',soup.html.body.find_all('h2')[0].text.strip()))
    
    #Get the total number of pages (up to 10)
    totalPages = min(math.ceil(totalJobOpenings/25),10)
    
    #Store the Final URL in a dictionary each state
    final_state_url_dict[url_state] = 'https://www.monster.com/jobs/search/?q=data-scientist&where='+url_state+'&stpage=1&page='+str(totalPages)

# Use the finalized URLs to get wrangle the job post text for each state

In [ ]:
#Access every URL with the find_all function
##print(soup.find_all('a'))
links = []
#Access every URL within the list of returned urls from findall
for link in soup.find_all('a'):
    if link.get('href')[0:21] == 'https://job-openings.': ## a job opening always begins with 'https://job-openings.monster.com'
        links.append(link.get('href'))        

#### Get job post URLs on first page

In [55]:
#Access every URL with the find_all function
##print(soup.find_all('a'))
links = []
#Access every URL within the list of returned urls from findall
for link in soup.find_all('a'):
    if link.get('href')[0:21] == 'https://job-openings.': ## a job opening always begins with 'https://job-openings.monster.com'
        links.append(link.get('href'))        

In [56]:
len(links)

250

In [57]:
#Get the total number of listed jobs 
totalJobOpenings = float(re.sub('[^0-9]','',soup.html.body.find_all('h2')[0].text.strip()))

In [58]:
# there will be a page for every 25 urls 
# for 1473 pages there are int(1473/25) pages
num_pages = str(min(10,int(totalJobOpenings/25)))
# Make a URL for up to the first 250 links


## Get ALL job url posts from the state (up to the first 250)

In [59]:
url = 'https://www.monster.com/jobs/search/?q=data-scientist&where='+url_state+'&stpage=1&page='+num_pages

#### Get job post URLs on first page

In [60]:
#Access every URL with the find_all function
##print(soup.find_all('a'))
links = []
#Access every URL within the list of returned urls from findall
for link in soup.find_all('a'):
    if link.get('href')[0:21] == 'https://job-openings.': ## a job opening always begins with 'https://job-openings.monster.com'
        links.append(link.get('href'))        

In [61]:
num_job_openings = len(links)

In [62]:
print( 'The number of job openings in {} are (at least): {}'.format(url_state,num_job_openings))

The number of job openings in NY are (at least): 250


# Check Each Link in State

In [63]:
count = 0

In [64]:
for job_opening_url in links:
    count += 1
    print(count)
    # sleep for .01 btw requests to not simulate ddos attack or w/e
    time.sleep(.01)
    
    #Access the website
    response = requests.get(job_opening_url)
    
    #Get the html soup!
    soup = BeautifulSoup(response.text, "html.parser")
    
    #Get the job title from the html soup
    url_job_title = soup.title.string[0:-14] # the last 14 characters are  '| Monster.com'
    
    #Get and clean the text from the job posting
    text = soup.body.find_all('script')[1]

    # Convert to string -> Convert to lower case 
    ## -> replace (all non ascii characters, ., and 3) with ','
    ### -> split string into a list separated by ','-> remove all consecutive duplicates 

    text_newest = [x[0] for x in groupby([x[0] for x in groupby(re.sub("[^a-zA-Z.+3]",',',str(text).lower()).split(','))])]

    #Remove the stopwords and empty spaces
    text_final = [x for x in text_newest if not x in stop_words and not x =='']
    
    #Add the values to the Dictionary
    addValsToDict(url_state,url_job_title)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [65]:
for x in dictionary:
    print(len(dictionary[x]))

250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250


In [66]:
#for x in dictionary:
#    print(len(dictionary[str(x)]))
data = pd.DataFrame(dictionary)

In [116]:
data.head(5)

,job_title,state,r,python,java,c++,ruby,perl,matlab,javascript,...,postgresql,nosql,sybase,rethinkdb,memcacheddb,couchdb,hbase,cassandra,mongodb,db2
0,"Data Scientist job at comScore, Inc.",NY,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Data Scientist - Predictive Modeling job at Re...,NY,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Data Scientist job at Xsell Resources,NY,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Client-Facing Applied Data Scientist job at Cy...,NY,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"Senior Data Scientist, Analytics Lab Team job ...",NY,1,1,1,1,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0


In [120]:
data.groupby('state').mean()

,r,python,java,c++,ruby,perl,matlab,javascript,scala,excel,...,postgresql,nosql,sybase,rethinkdb,memcacheddb,couchdb,hbase,cassandra,mongodb,db2
state,,,,,,,,,,,,,,,,,,,,,
NY,0.556,0.66,0.248,0.104,0.024,0.048,0.088,0.048,0.164,0.128,...,0.036,0.124,0.004,0.0,0.0,0.004,0.048,0.056,0.052,0.0
